In [1]:
import pandas as pd
import tushare as ts
from sqlalchemy import create_engine
import time
import datetime
import json
import numpy as np

In [2]:
# 导入token
with open('./parameters.json','r') as f:
    p=json.load(f)
    user = p['user']
    port = p['port']
    psw = p['password']
    host = p['host']
    token=p['TU_share_pro_taken']
    cnnstr = "mysql://" + user + ":" + psw + "@" + host + ":" + str(port) + "/stock?charset=utf8&use_unicode=1"
engine_ts = create_engine(cnnstr)
ts.set_token(token)

In [71]:
# ----------参数------------------
path_save = './DataAnalysis/many_stock/'
stocklist=list(pd.read_csv('./DataAnalysis/stock_list.csv').iloc[:,0])
start='20060101'
end='20201125'
backwards = 66
foresee = 5
max_pe=100*1000
max_pb=100*1000
max_mv=100*1000*1000*1000
split=0.8

In [4]:
# 读取宏观数据
Marco=pd.read_csv('./DataAnalysis/Macro2016_20201031.csv')

In [5]:
# 处理宏观数据
df3=Marco
## 处理将所有值的部分除以100
df3=df3.set_index('YYYYMMDD')
df3=df3.apply(lambda x: x/100)
df3.reset_index(inplace=True)

In [6]:
df3.head(5)

,YYYYMMDD,nt_yoy,gdp_yoy,m2_yoy,ppi_yoy,1y
0,20060101,0.0189,0.124,0.1921,0.0305,NaN
1,20060102,0.0189,0.124,0.1921,0.0305,NaN
2,20060103,0.0189,0.124,0.1921,0.0305,NaN
3,20060104,0.0189,0.124,0.1921,0.0305,NaN
4,20060105,0.0189,0.124,0.1921,0.0305,NaN


In [7]:
# daily = ts.pro_bar(ts_code=stocklist[0], start_date=start, end_date=end,
#                             asset='E', adj='qfq', freq='D')
# daily.sort_values(by='trade_date',inplace=True)

In [8]:
# pro = ts.pro_api()
# daily_basic=pro.daily_basic(ts_code=stocklist[0], start_date=start, end_date=end, 
#                             fields='ts_code,trade_date,turnover_rate,pe_ttm,pb,total_mv')
# daily_basic.sort_values(by='trade_date',inplace=True)

In [9]:
# df1=daily
# df1.reset_index(drop=True,inplace=True)
# df1.head(5)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20060104,1.3809,1.4259,1.3786,1.4147,1.3831,0.0316,2.2847,154450.68,96989.0066
1,000001.SZ,20060105,1.4192,1.4304,1.4056,1.4237,1.4147,0.0090,0.6362,153855.86,96812.5280
2,000001.SZ,20060106,1.4417,1.4642,1.4304,1.4439,1.4237,0.0202,1.4188,238663.99,153652.2134
3,000001.SZ,20060109,1.4417,1.4507,1.4282,1.4394,1.4439,-0.0045,-0.3117,110221.01,70422.2378
4,000001.SZ,20060110,1.4304,1.4327,1.3854,1.4147,1.4394,-0.0247,-1.7160,214267.43,133992.6258


In [10]:
# df1['price_change']=(df1['close']-df1['pre_close'])/df1['pre_close']
# df1.drop(columns=['vol','amount','change','pct_chg'],inplace=True)
# df1.tail(5)

,ts_code,trade_date,open,high,low,close,pre_close,price_change
3483,000001.SZ,20201119,18.59,18.98,18.30,18.85,18.46,0.021127
3484,000001.SZ,20201120,18.83,18.99,18.52,18.86,18.85,0.000531
3485,000001.SZ,20201123,18.85,19.62,18.80,19.62,18.86,0.040297
3486,000001.SZ,20201124,19.62,19.68,19.17,19.36,19.62,-0.013252
3487,000001.SZ,20201125,19.48,19.70,19.05,19.06,19.36,-0.015496


In [11]:
# df2=daily_basic
# df2.tail(5)

,ts_code,trade_date,turnover_rate,pe_ttm,pb,total_mv
4,000001.SZ,20201119,0.6244,13.5623,1.2665,3.658016e+07
3,000001.SZ,20201120,0.3473,13.5695,1.2672,3.659956e+07
2,000001.SZ,20201123,0.6008,14.1163,1.3182,3.807441e+07
1,000001.SZ,20201124,0.3497,13.9292,1.3008,3.756986e+07
0,000001.SZ,20201125,0.2848,13.7134,1.2806,3.698768e+07


In [12]:
# 处理指标，都除以最大数字
# df2['pe_ttm']=df2.pe_ttm.apply(lambda x: x/max_pe)
# df2['pb']=df2.pb.apply(lambda x: x/max_pb)
# df2['total_mv']=df2.total_mv.apply(lambda x: x/max_mv)
# df2.tail(5)

,ts_code,trade_date,turnover_rate,pe_ttm,pb,total_mv
4,000001.SZ,20201119,0.6244,0.000136,0.000013,0.000366
3,000001.SZ,20201120,0.3473,0.000136,0.000013,0.000366
2,000001.SZ,20201123,0.6008,0.000141,0.000013,0.000381
1,000001.SZ,20201124,0.3497,0.000139,0.000013,0.000376
0,000001.SZ,20201125,0.2848,0.000137,0.000013,0.000370


In [13]:
# df=pd.merge(df1,df2,how='left',on=['ts_code','trade_date'],left_index=False,right_index=False,copy=True)
# df['trade_date']=df['trade_date'].astype('int64')
# df.tail(5)

,ts_code,trade_date,open,high,low,close,pre_close,price_change,turnover_rate,pe_ttm,pb,total_mv
3483,000001.SZ,20201119,18.59,18.98,18.30,18.85,18.46,0.021127,0.6244,0.000136,0.000013,0.000366
3484,000001.SZ,20201120,18.83,18.99,18.52,18.86,18.85,0.000531,0.3473,0.000136,0.000013,0.000366
3485,000001.SZ,20201123,18.85,19.62,18.80,19.62,18.86,0.040297,0.6008,0.000141,0.000013,0.000381
3486,000001.SZ,20201124,19.62,19.68,19.17,19.36,19.62,-0.013252,0.3497,0.000139,0.000013,0.000376
3487,000001.SZ,20201125,19.48,19.70,19.05,19.06,19.36,-0.015496,0.2848,0.000137,0.000013,0.000370


In [14]:
# df=pd.merge(df,df3,how='left',left_on='trade_date',right_on='YYYYMMDD',left_index=False,right_index=False,copy=True)
# df.tail(5)

,ts_code,trade_date,open,high,low,close,pre_close,price_change,turnover_rate,pe_ttm,pb,total_mv,YYYYMMDD,nt_yoy,gdp_yoy,m2_yoy,ppi_yoy,1y
3483,000001.SZ,20201119,18.59,18.98,18.30,18.85,18.46,0.021127,0.6244,0.000136,0.000013,0.000366,NaN,NaN,NaN,NaN,NaN,NaN
3484,000001.SZ,20201120,18.83,18.99,18.52,18.86,18.85,0.000531,0.3473,0.000136,0.000013,0.000366,NaN,NaN,NaN,NaN,NaN,NaN
3485,000001.SZ,20201123,18.85,19.62,18.80,19.62,18.86,0.040297,0.6008,0.000141,0.000013,0.000381,NaN,NaN,NaN,NaN,NaN,NaN
3486,000001.SZ,20201124,19.62,19.68,19.17,19.36,19.62,-0.013252,0.3497,0.000139,0.000013,0.000376,NaN,NaN,NaN,NaN,NaN,NaN
3487,000001.SZ,20201125,19.48,19.70,19.05,19.06,19.36,-0.015496,0.2848,0.000137,0.000013,0.000370,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 纵向填充，用后面的值填充前面，用以补充最早缺失的数据/数据排序为从早到晚
# df.fillna(method='bfill',axis=0,inplace=True) 
# 纵向填充，用前面的值填充后面，用以补充最近缺失的数据/数据排序为从早到晚
# df.fillna(method='ffill',axis=0,inplace=True) 

In [16]:
# df.drop(columns=['YYYYMMDD'],inplace=True)
# df.tail(5)

,ts_code,trade_date,open,high,low,close,pre_close,price_change,turnover_rate,pe_ttm,pb,total_mv,nt_yoy,gdp_yoy,m2_yoy,ppi_yoy,1y
3483,000001.SZ,20201119,18.59,18.98,18.30,18.85,18.46,0.021127,0.6244,0.000136,0.000013,0.000366,0.005,0.007,0.105,-0.021,0.0318
3484,000001.SZ,20201120,18.83,18.99,18.52,18.86,18.85,0.000531,0.3473,0.000136,0.000013,0.000366,0.005,0.007,0.105,-0.021,0.0318
3485,000001.SZ,20201123,18.85,19.62,18.80,19.62,18.86,0.040297,0.6008,0.000141,0.000013,0.000381,0.005,0.007,0.105,-0.021,0.0318
3486,000001.SZ,20201124,19.62,19.68,19.17,19.36,19.62,-0.013252,0.3497,0.000139,0.000013,0.000376,0.005,0.007,0.105,-0.021,0.0318
3487,000001.SZ,20201125,19.48,19.70,19.05,19.06,19.36,-0.015496,0.2848,0.000137,0.000013,0.000370,0.005,0.007,0.105,-0.021,0.0318


In [72]:
# 处理单片数据
def data_calculation(df):   # df 为数据框格式
    dfa=df.copy()
    dfa.reset_index(drop=True,inplace=True)   # 重新索引
    dfa.insert(dfa.shape[1],'factor',np.nan)  #  增加复权列
    dfa.loc[0,'factor']=1   #  设置起始复权值为1
    for i in range(1,len(dfa)):   #  循环计算后续复权值
        dfa.loc[i,'factor']=dfa.loc[i-1,'factor']*(1+dfa.loc[i,'price_change'])
    # 重新计算开高低收的值
    dfa['open2']=dfa['open']/dfa['close']*dfa['factor']
    dfa['high2']=dfa['high']/dfa['close']*dfa['factor']
    dfa['low2']=dfa['low']/dfa['close']*dfa['factor']   
    ts_code=dfa.loc[0,'ts_code']
    trade_date=dfa.loc[backwards-1,'trade_date']
    dfa.drop(columns=['ts_code','trade_date','open','high','low','close','pre_close','price_change'],inplace=True)  # 删除多余列
    #  计算标签值
    label=dfa.loc[backwards:backwards+foresee-1,'factor'].max()/dfa.loc[backwards-1,'factor']-1
    dfa.insert(dfa.shape[1],'label_value',label)  
    # 删除尾部5行，确保X值在100
    dfa.drop(dfa.tail(foresee).index,inplace=True)
    #  返回数据框以及标签值
    return dfa, ts_code,trade_date

In [73]:
# 循环存入单片数据
# for i in range(len(df)-backwards-foresee):
#     ddd,ts_code,trade_date = data_calculation(df[i:i+backwards+foresee])
#     ddd.to_csv(path_or_buf='./DataAnalysis/many_stock/'+ts_code+'_'+str(trade_date)+'.csv',sep=',',index=False)
#     print(f'stock:{ts_code},处理完毕：{round(i/(len(df)-backwards-foresee)*100,2)} %')

stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：0.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
stock:000001.SZ,处理完毕：1.0 %
s

stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：9.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.SZ,处理完毕：10.0 %
stock:000001.

stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕：18.0 %
stock:000001.SZ,处理完毕

stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：26.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕：27.0 %
stock:000001.SZ,处理完毕

stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：35.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕：36.0 %
stock:000001.SZ,处理完毕

...

stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：61.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕：62.0 %
stock:000001.SZ,处理完毕

stock:000001.SZ,处理完毕：69.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕：70.0 %
stock:000001.SZ,处理完毕

stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：78.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕：79.0 %
stock:000001.SZ,处理完毕

stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：87.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕：88.0 %
stock:000001.SZ,处理完毕

stock:000001.SZ,处理完毕：95.0 %
stock:000001.SZ,处理完毕：95.0 %
stock:000001.SZ,处理完毕：95.0 %
stock:000001.SZ,处理完毕：95.0 %
stock:000001.SZ,处理完毕：95.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕：96.0 %
stock:000001.SZ,处理完毕

In [ ]:
def individual_stock(ts_code):
    # 获取股票价格历史
    daily = ts.pro_bar(ts_code=ts_code, start_date=start, end_date=end,
                            asset='E', adj='qfq', freq='D')
    daily.sort_values(by='trade_date',inplace=True)
    # 获取股票日KPI
    pro = ts.pro_api()
    daily_basic=pro.daily_basic(ts_code=ts_code, start_date=start, end_date=end, 
                            fields='ts_code,trade_date,turnover_rate,pe_ttm,pb,total_mv')
    daily_basic.sort_values(by='trade_date',inplace=True)
    # 处理价格历史
    df1=daily
    df1.reset_index(drop=True,inplace=True)
    df1['price_change']=(df1['close']-df1['pre_close'])/df1['pre_close']
    df1.drop(columns=['vol','amount','change','pct_chg'],inplace=True)
    # 处理日KPI
    df2=daily_basic
    ## 处理指标，都除以最大数字
    df2['pe_ttm']=df2.pe_ttm.apply(lambda x: x/max_pe)
    df2['pb']=df2.pb.apply(lambda x: x/max_pb)
    df2['total_mv']=df2.total_mv.apply(lambda x: x/max_mv)
    # 合并数据
    df=pd.merge(df1,df2,how='left',on=['ts_code','trade_date'],left_index=False,right_index=False,copy=True)
    df['trade_date']=df['trade_date'].astype('int64')
    # 合并宏观数据
    df=pd.merge(df,df3,how='left',left_on='trade_date',right_on='YYYYMMDD',left_index=False,right_index=False,copy=True)
    # 处理合并后的数据
    df.fillna(method='bfill',axis=0,inplace=True) # 纵向填充，用后面的值填充前面，用以补充最早缺失的数据/数据排序为从早到晚
    df.fillna(method='ffill',axis=0,inplace=True) # 纵向填充，用前面的值填充后面，用以补充最近缺失的数据/数据排序为从早到晚
    df.drop(columns=['YYYYMMDD'],inplace=True)
    # 循环存入单片数据
    for i in range(len(df)-backwards-foresee):
        ddd,ts_code,trade_date = data_calculation(df[i:i+backwards+foresee])
        ddd.to_csv(path_or_buf=path_save+ts_code+'_'+str(trade_date)+'.csv',sep=',',index=False)
        print(f'stock:{ts_code},处理完毕：{round(i/(len(df)-backwards-foresee)*100,2)} %')
    print(f'stock:{ts_code}处理完毕！')